In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('pfm_train.csv')
test_df = pd.read_csv('pfm_test.csv')

In [3]:
train_df.head(5)
y_train = train_df.Attrition
train_df.drop(['Attrition'],axis=1,inplace=True)

In [4]:
def cleaningdata(data):
    data.drop(['Over18'],axis=1,inplace = True)
    data.drop(['StandardHours'],axis=1,inplace = True)
    data.drop(['EmployeeNumber'],axis=1,inplace = True)
    data = pd.get_dummies(data)
    return data

In [5]:
train_df = cleaningdata(train_df)
test_df = cleaningdata(test_df)

In [6]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(train_df, y_train, test_size=0.2)

In [7]:
from sklearn import ensemble
m3 = ensemble.ExtraTreesClassifier(n_estimators=40, 
                                   max_depth=20,
                                   min_samples_split=3,
                                   random_state=0)
m3.fit(train_df, y_train)
y_pred3 = m3.predict(test_df)
sub = pd.DataFrame()
sub['result1'] = y_pred3
# print('Accuracy: {}'.format(m3.score(X_test,y_test)))

In [8]:
from sklearn import ensemble
m4 = ensemble.RandomForestClassifier(n_estimators=100, 
                                   max_depth=22,
                                   min_samples_split=3,
                                   random_state=0)
m4.fit(train_df, y_train)
y_pred4 = m4.predict(test_df)

sub['result2'] = y_pred4

# print('Accuracy: {}'.format(m4.score(X_test,y_test)))

In [9]:
%%time
from lightgbm import LGBMClassifier
m1 = LGBMClassifier(class_weight='balanced', boosting_type='dart',
                         drop_rate=0.1, min_data_in_leaf=50, 
                         max_bin=255,
                         n_estimators=800,
                         bagging_fraction=0.01,
                         min_sum_hessian_in_leaf=1,
                         importance_type='gain',
                         learning_rate=0.2, 
                         max_depth=10, 
                         num_leaves=10)
m1.fit(train_df, y_train)
y_pred1 = m1.predict(test_df)
sub['result3'] = y_pred1

# print('Accuracy: {}'.format(m1.score(X_test,y_test)))

Wall time: 561 ms


c:\users\user\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [10]:
preds = sub.mode(axis=1)

In [11]:
sub = pd.DataFrame(columns=['result'])

In [12]:
sub['result'] = preds[0]
sub.to_csv('3ensemble_output.csv',index=None)